In [1]:
import os, json, PIL
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

In [2]:
class Statistics(Dataset):
    def __init__(self):
        train_dir = './train/'
        test_dir = './test/'
        
        self.imgs = []
        for num in sorted(os.listdir(train_dir)):
            with open(train_dir + '{}/{}.json'.format(num, num), 'r') as j:
                temp = json.load(j)
                for info in temp['annotations']:
                    self.imgs.append(train_dir + '{}/{}.png'.format(num, info['image_id']))
                    
        for num in sorted(os.listdir(test_dir)):
            with open(test_dir + '{}/{}.json'.format(num, num), 'r') as j:
                temp = json.load(j)
                for info in temp['annotations']:
                    self.imgs.append(test_dir + '{}/{}.png'.format(num, info['image_id']))

    def __getitem__(self, idx):
        img_path = self.imgs[idx]
        img = PIL.Image.open(img_path).convert('RGB')
        return transforms.ToTensor()(img)

    def __len__(self):
        return len(self.imgs)

In [3]:
dataset = Statistics()
loader = DataLoader(dataset, batch_size=64, shuffle=False)

mean = 0.
std = 0.
nb_samples = 0.
for data in loader:
    batch_samples = data.size(0)
    data = data.view(batch_samples, data.size(1), -1)
    mean += data.mean(2).sum(0)
    std += data.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples
print(mean, std)

tensor([0.3005, 0.3035, 0.2712]) tensor([0.1519, 0.1252, 0.0996])
